In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import pyfpgrowth
from mlxtend.frequent_patterns import apriori 
from mlxtend.frequent_patterns import association_rules

In [18]:
bakery=pd.read_csv('BreadBasket_DMS.csv')
bakery['Item'].value_counts()

Coffee            5471
Bread             3325
Tea               1435
Cake              1025
Pastry             856
                  ... 
Olum & polenta       1
The BART             1
Polenta              1
Adjustment           1
Gift voucher         1
Name: Item, Length: 95, dtype: int64

In [19]:
bakery.shape

(21293, 4)

In [20]:
bakery.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [21]:
Items={}
cnt=0
for item in bakery['Item']:
    if item in Items:
        Items[item]=Items[item]+1
    else:
        Items[item]=1
        cnt+=1

In [22]:
#min_support 50% so
min_support=0.5
support_cnt=0.5*cnt
support_cnt=int(support_cnt)
support_cnt

47

In [23]:
bakery['Set_bit']= 1

In [24]:
bakery.head()

,Date,Time,Transaction,Item,Set_bit
0,2016-10-30,09:58:11,1,Bread,1
1,2016-10-30,10:05:34,2,Scandinavian,1
2,2016-10-30,10:05:34,2,Scandinavian,1
3,2016-10-30,10:07:57,3,Hot chocolate,1
4,2016-10-30,10:07:57,3,Jam,1


In [25]:
basket = bakery.groupby(['Transaction', 'Item'])['Set_bit'].sum().unstack().fillna(0)
basket.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def one_not_code_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(one_not_code_units)

print(basket_sets)

Item         Adjustment  Afternoon with the baker  Alfajores  Argentina Night  \
Transaction                                                                     
1                     0                         0          0                0   
2                     0                         0          0                0   
3                     0                         0          0                0   
4                     0                         0          0                0   
5                     0                         0          0                0   
...                 ...                       ...        ...              ...   
9680                  0                         0          0                0   
9681                  0                         0          0                0   
9682                  0                         0          0                0   
9683                  0                         0          0                0   
9684                  0     

In [27]:
frequent_itemsets = apriori(basket_sets,min_support=0.05,  use_colnames=True)
print(frequent_itemsets)


     support         itemsets
0   0.324940          (Bread)
1   0.103137           (Cake)
2   0.475081         (Coffee)
3   0.054034        (Cookies)
4   0.057916  (Hot chocolate)
5   0.061379      (Medialuna)
6   0.079005           (NONE)
7   0.085510         (Pastry)
8   0.071346       (Sandwich)
9   0.141643            (Tea)
10  0.089393  (Coffee, Bread)
11  0.054349   (Cake, Coffee)


In [28]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667
1,(Coffee),(Cake),0.475081,0.103137,0.054349,0.114399,1.109196,0.00535,1.012717
